In [ ]:
!pip install livelossplot

**import all libraries**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import keras
from keras.preprocessing import image
from keras.engine import Layer
from keras.layers import Conv2D, Conv3D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
from time import time
import numpy as np
import os
import random
import tensorflow as tf
from PIL import Image, ImageFile
import cv2
from livelossplot import PlotLossesKeras

**import all the data**

In [ ]:
images_gray = np.load('/kaggle/input/image-colorization/l/gray_scale.npy')
images_lab = np.load('/kaggle/input/image-colorization/ab/ab/ab1.npy')

In [ ]:
image_gray = images_gray[0]
image_lab = images_lab[0]

In [ ]:

# Initializing with zeros ( or any random number)
img = np.zeros((224, 224, 3))
img[:, :, 0] = image_gray
img[:, :, 1:] = image_lab

# Changing the data type of the img array to 'uint8'
# Refer the above markdown for it.
img = img.astype('uint8')

In [ ]:
img_ = cv2.cvtColor(img, cv2.COLOR_LAB2RGB)


print('Gray scale image')
plt.imshow(image_gray)
plt.show()

print('Recreated image')
plt.imshow(img_)
plt.show()

In [ ]:
X=[]
Y=[]
for i in range(0,2000):
    image_gray = images_gray[i]
    image_lab = images_lab[i]
    img = np.zeros((224, 224, 3))
    img[:, :, 0] = image_gray
    img[:, :, 1:] = image_lab
    img = img.astype('uint8')
    img_ = cv2.cvtColor(img, cv2.COLOR_LAB2RGB)
    new_array=cv2.resize(img_,(256,256))
    new_array = new_array.astype('float32')
    new_array /= 255
    lab = rgb2lab(new_array)
    try:
        lab = rgb2lab(new_array)
        X.append(lab[:,:,0])
        Y.append(lab[:,:,1:] / 128)
    except:
        print('error')
    

In [ ]:
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,))
print(X.shape)
print(Y.shape)

**model******

In [ ]:

encoder_input1 = Input(shape=(256, 256, 1,))
encoder_output2 = Conv2D(64, (3,3),strides=(2, 2) ,activation='relu', padding='same')(encoder_input1)
encoder_output3 = Conv2D(128, (3,3),strides=(1, 1), activation='relu', padding='same')(encoder_output2)
encoder_output4 = Conv2D(128, (3,3), strides=(2, 2),activation='relu', padding='same')(encoder_output3)
encoder_output5 = Conv2D(256, (3,3),strides=(1, 1), activation='relu', padding='same')(encoder_output4)
encoder_output6 = Conv2D(256, (3,3),strides=(2, 2), activation='relu', padding='same')(encoder_output5)
encoder_output7 = Conv2D(512, (3,3), strides=(1, 1),activation='relu', padding='same')(encoder_output6)
encoder_output8 = Conv2D(512, (3,3), strides=(1, 1),activation='relu', padding='same')(encoder_output7)
encoder_output9 = Conv2D(256, (3,3),strides=(1, 1) ,activation='relu', padding='same')(encoder_output8)

#Decoder
decoder_output10 = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output9)
skip_4_drop = Dropout(0.25)(encoder_output6)
skip_4 = concatenate([decoder_output10, encoder_output6])
decoder_output11 = UpSampling2D((2, 2))(skip_4)
decoder_output12 = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output11)
skip_5_drop = Dropout(0.25)(encoder_output4)
skip_5 = concatenate([decoder_output12, encoder_output4])
decoder_output13 = UpSampling2D((2, 2))(skip_5)
decoder_output14 = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output13)
decoder_output15 = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output14)
decoder_output16 = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output15)
decoder_output17 = UpSampling2D((2, 2))(decoder_output16)
model = Model(inputs=encoder_input1, outputs=decoder_output17)

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, 'model_diagram.png')
plt.figure(figsize=(30, 30))
plt.imshow(Image.open('model_diagram.png'))

In [ ]:
Callbacks=[
    PlotLossesKeras()
    
]
print(model.summary())
model.compile(optimizer='adam', loss='mse' , metrics=['accuracy'])
t=model.fit(X,Y,validation_split=0.2,batch_size=90, epochs=20,verbose=1,callbacks=Callbacks)
print(t.history.keys())

In [ ]:
test = '../input/manqwe/rames.jpg'
#test = os.listdir(test_path)
for imgName in test:
    color_me = []
    img = img_to_array(load_img(test))
    img = resize(img ,(256,256))
    color_me.append(img)
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))
print(color_me.shape)
output = model.predict(color_me)
output = output * 128
# Output colorizations
for i in range(len(output)):
    result = np.zeros((256, 256, 3))
    result[:,:,0] = color_me[i][:,:,0]
    result[:,:,1:] = output[i]
    imsave("result1.png", lab2rgb(result))
    x=lab2rgb(result)
    plt.imshow(x)
    plt.show()